In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Carregar os dados tratados
df = pd.read_csv("../Data/Stroke.csv")

# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Pipeline com RFECV para seleção de features e RandomForest
pipeline = Pipeline([
    ('rfecv', RFECV(estimator=RandomForestClassifier(random_state=42, n_estimators=50, max_depth=None, criterion='gini'), 
                    step=1, cv=5)),  # implementando RFECV
    ('rf', RandomForestClassifier(random_state=42, n_estimators=50, max_depth=None, criterion='gini'))
])

# Ajustando o modelo no conjunto de treinamento
pipeline.fit(X_train, y_train)

# Resultados
print(f"\nNúmero de features selecionadas: {pipeline.named_steps['rfecv'].n_features_}")
print(f"Melhores features selecionadas: {X.columns[pipeline.named_steps['rfecv'].support_]}")

# Avaliação final no conjunto de teste
y_pred = pipeline.predict(X_test)

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))



Número de features selecionadas: 15
Melhores features selecionadas: Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Male', 'ever_married_No', 'ever_married_Yes',
       'work_type_Private', 'work_type_Self-employed', 'Residence_type_Rural',
       'Residence_type_Urban', 'smoking_status_Unknown',
       'smoking_status_formerly smoked', 'smoking_status_never smoked'],
      dtype='object')

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       951
           1       0.98      1.00      0.99       943

    accuracy                           0.99      1894
   macro avg       0.99      0.99      0.99      1894
weighted avg       0.99      0.99      0.99      1894


Matriz de Confusão:
[[928  23]
 [  0 943]]


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

# Carregar os dados tratados
df = pd.read_csv("../Data/Stroke.csv")

# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Configurando o modelo RandomForest com os melhores parâmetros
rf_model = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=None, criterion='gini')

# Loop para rodar SelectKBest para cada valor de k
for k in range(1, X_train.shape[1] + 1):  # De 1 até 18 (total de features)
    print(f"\nAvaliação com k = {k} (Seleção das {k} melhores features):")
    
    # SelectKBest para o valor de k
    skb = SelectKBest(score_func=f_classif, k=k)
    X_train_k = skb.fit_transform(X_train, y_train)
    X_test_k = skb.transform(X_test)

    # Treinando o modelo com as k melhores features
    rf_model.fit(X_train_k, y_train)
    
    # Fazendo as previsões
    y_pred = rf_model.predict(X_test_k)
    
    # Imprimindo o classification report
    print(classification_report(y_test, y_pred))



Avaliação com k = 1 (Seleção das 1 melhores features):
              precision    recall  f1-score   support

           0       0.84      0.69      0.76       951
           1       0.73      0.86      0.79       943

    accuracy                           0.78      1894
   macro avg       0.79      0.78      0.78      1894
weighted avg       0.79      0.78      0.78      1894


Avaliação com k = 2 (Seleção das 2 melhores features):
              precision    recall  f1-score   support

           0       0.87      0.68      0.76       951
           1       0.73      0.90      0.81       943

    accuracy                           0.79      1894
   macro avg       0.80      0.79      0.79      1894
weighted avg       0.80      0.79      0.79      1894


Avaliação com k = 3 (Seleção das 3 melhores features):
              precision    recall  f1-score   support

           0       0.87      0.68      0.76       951
           1       0.73      0.90      0.81       943

    accuracy  